In [24]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
import dotenv 
from openai import OpenAI
from tqdm import tqdm
import requests
import re
from bs4 import BeautifulSoup
from joblib import Parallel, delayed, Memory
import backoff
from markdownify import markdownify as md
from IPython.display import display, Markdown, Latex
from tqdm import tqdm
import pandas as pd
import uusikielemme

assert dotenv.load_dotenv('.env')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
# Instead of lru_cache, we have to use joblib.Memory, as per https://github.com/joblib/joblib/issues/226
mem = Memory('.joblib_cache')
requests.get = mem.cache(requests.get, verbose=False)

In [39]:
import requests
from bs4 import BeautifulSoup

def fetch_non_xml_urls_from_sitemap(sitemap_url):
    response = requests.get(sitemap_url)
    if response.status_code == 200:
        print()
        soup = BeautifulSoup(response.text, 'html.parser')
        urls = [loc.text for loc in soup.find_all('loc')]
        sitemap_urls = [url for url in urls if url.endswith('.xml')]
        other_urls = [url for url in urls if not url.endswith('.xml')]
        for sitemap in sitemap_urls:
            other_urls.extend(fetch_non_xml_urls_from_sitemap(sitemap))
        return other_urls
    else:
        print(f"Failed to fetch sitemap from {sitemap_url}")
        return []

sitemap_url = "https://uusikielemme.fi/sitemap_index.xml"
non_xml_urls = fetch_non_xml_urls_from_sitemap(sitemap_url)

In [41]:
len(non_xml_urls)

893

In [46]:
from uusikielemme.utils import extract_post_content

data = Parallel(64)(
    delayed(extract_post_content(url)['content'])
    for url in tqdm(non_xml_urls)
)
# for url in non_xml_urls:
#     display(Markdown())
#     break

  0%|          | 0/893 [00:00<?, ?it/s]

 10%|█         | 92/893 [00:59<08:35,  1.55it/s]Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe9984f6f90>>
Traceback (most recent call last):
  File "/home/tornikeo/micromamba/envs/scrape/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
 12%|█▏        | 109/893 [01:10<10:27,  1.25it/s]

In [7]:
display(data[2]['category'])
display(Markdown(data[2]['posts'][0]['content'][:1090]))

'Authentic Language Use'

# Stretching – Yoga in Finnish – Jooga suomeksi 

In this article, you can find some vocabulary and phrases for yoga in Finnish. I’m basing this article on a video by [Ninan jooga](https://www.youtube.com/channel/UCQcjlVqbs8wmyOe1u-NZBRQ). I’ve picked a small part of [this video](https://youtu.be/-tc86UJlWR0?t=32) to analyze. I specifically picked the part of the video that deals with stretching left and right over your head. It’s a fairly simple thing to do, so perfect to start with. If you are interested in more of this type of content, let me know in the comments!



You can also check out the general article with [yoga vocabulary](https://uusikielemme.fi/finnish-vocabulary/vocabulary-lists/jooga-finnish-yoga-vocabulary-poses-and-verbs/) to learn more words!



The part of the video we will be looking at starts at 00:32 and ends at 3:06. It’s the part where Nina is stretching her back to the left and the right.





Below, you can find the transcription of the part of the video we will be looking at today. Underlined are the spoken language elements Nina uses in her vide

In [10]:
data[0]

{'category': 'Adjectives',
 'posts': [{'title': 'Adjectives and opposites – B1.1 level adjectives – YKI-testi',
   'text': '# Adjectives and opposites – B1.1 level adjectives – YKI-testi \n\nThis article contains adjectives and their opposites. I have a similar article for complete beginners already, where the [most simple adjectives](https://uusikielemme.fi/finnish-vocabulary/word-types/adjectives/basic-adjectives-and-opposites) are listed. This article expands on that topic, with B1.1 level adjectives. This doesn’t mean that these adjectives are all equally hard, but they would be very useful to learn if you’re going to the YKI-testi and want to prepare with more vocabulary.\n\n\n\nI’ve excluded all adjective pairings where one of the two opposites contains [the prefix epä-](https://uusikielemme.fi/finnish-vocabulary/words-ending-in/finnish-prefix-epa-for-negation-non-un) or [the suffix -ton](https://uusikielemme.fi/finnish-vocabulary/words-ending-in/words-ending-in-ton). I have sepa

In [33]:
import genanki
import random

unique_model_id = random.randrange(1 << 30, 1 << 31)
unique_deck_id = random.randrange(1 << 30, 1 << 31)

In [34]:
model = genanki.Model(
  unique_model_id,
  'Simple model',
  fields=[
    {'name': 'Question'},
    {'name': 'Answer'},
  ],
  templates=[
    {
      'name': 'Card 1',
      'qfmt': '{{Question}}',
      'afmt': '{{FrontSide}}<hr id="answer">{{Answer}}',
    },
  ])

In [35]:
deck = genanki.Deck(unique_deck_id, 'Uusikielemme')

posts = []
for category in data: # List of dicts
    # print(category['category'])
    for post in category['posts']: # List of dicts
        if post['content']:
            # posts.append(f"{category['category']} - {post['title']}")
            posts.append(post['content'])

            # print(post)
            # display(post['content'])
            # if random.randint(0, 100) < 5:
            # display(Markdown(post['content'][:1000]))
            # lim -= 1
            # if not lim: return

            # for table in post['tables']: # List of dataframes
            #     display(table)
                    # return
                    # for 
                    # note = genanki.Note(model=model, fields=['Capital of Argentina', 'Buenos Aires'])
                    # display(table)
                    # break
                    # return
        # print(category['posts'])
        # break

In [29]:
len(posts)

244

In [36]:
# print('\n'.join(posts))
# print(posts[0])

In [29]:

deck.add_note(note)
genanki.Package(deck).write_to_file('output.apkg')

In [30]:
# i = 0
# for cat in data:
#     for page in cat:
#         display(Markdown(page['content']))
#         i += 1
#         if i > 10: break

In [ ]:
genanki.Note(
    model = model,
    fields=[]
)